Calculating NQR frequencies
===========================

In this, we will demonstrate how to calculate NQR frequencies from an ab-initio EFG calculation. This is based on the nqr.py script, which implements a command-line utility to perform the below.

You can use this utility from the command line like:

```bash
> nqr.py [species] [directory]
```

e.g. if you're in a calculation directory and you want the chlorine couplings you do

```bash
> nqr.py Cl .
```

Definition
----------

From "Nuclear Quadrupole Resonance Spectroscopy" by Hand and Das, Solid State Physics Supplement 1, the NQR transition frequency $\omega$ for a spin $S$ with electric field gradient principal component $V_{zz}$ and quadrupole moment $Q$ is


$\begin{align}
    A &= \frac{eV_{zz}Q}{4S(2S-1)}\\
    \omega &= 3 A (2 |m| + 1)/\hbar
\end{align}$

for all available transitions $m \rightarrow m+1$ where $-S \le m \le S$.


The calculation
---------------

We do an EFG calculation using CASTEP with the following input files:

### NaClO3.param

```raw
xcfunctional: PBE 
opt_strategy: speed
task: magres
magres_task: efg
cut_off_energy: 50 ry
```

### NaClO3.cell

```raw
%block SPECIES_POT
Pb  3|2.4|2.35|1.6|9.2|12.9|16.5|60UU:61UU:52UU[]
%endblock SPECIES_POT

%block POSITIONS_ABS
ang
O 2.664659 0.007779 1.298116
O 3.295805 1.990002 3.910951
O 0.007779 1.298116 2.664659
O 1.298116 2.664659 0.007779
O 4.585891 0.622991 6.567871
O 1.990002 3.910951 3.295805
O 3.910944 3.295798 1.990008
O 3.280141 5.277909 5.952599
O 0.622985 6.567871 4.585885
O 5.277909 5.952599 3.280141
O 6.567871 4.585885 0.622991
O 5.952599 3.280141 5.277909
Na 0.445438 0.445438 0.445438
Na 3.733280 2.842423 6.130271
Na 2.842423 6.130271 3.733280
Na 6.130271 3.733280 2.842423
Cl 3.833830 6.030029 0.545995
Cl 2.742024 2.742024 2.742024
Cl 0.545995 3.833823 6.030023
Cl 6.030029 0.545995 3.833830
%endblock POSITIONS_ABS

%block LATTICE_CART
ang
6.575801 0.000000 0.000000
0.000000 6.575801 0.000000
0.000000 0.000000 6.575801
%endblock LATTICE_CART

kpoint_mp_grid 1 1 1
kpoint_mp_offset 0.25 0.25 0.25
```

and, if just running locally rather than using a cluster submission system, run like

```bash
> castep.serial NaClO3
```

Code
----

First, we need to make a few imports to get the magres atoms datastructure, constants for units conversions and maths libraries.

In [1]:
from __future__ import print_function

from magres.atoms import MagresAtoms
from magres.constants  import millibarn, megahertz
import numpy, math

Now, let's define a function that will tell us the starting *m* for all available transitions *m->m+1* for a particular nuclear spin *S*.

In [2]:
def available_m(spin):
     return [m for m in numpy.arange(-spin, spin+1, 1) if m >= 0.0][:-1]

print ("S=0.5, available |m| = ", available_m(0.5))
print ("S=1.0, available |m| = ", available_m(1.0))
print ("S=1.5, available |m| = ", available_m(1.5))
print ("S=2.0, available |m| = ", available_m(2.0))

S=0.5, available |m| =  []
S=1.0, available |m| =  [0.0]
S=1.5, available |m| =  [0.5]
S=2.0, available |m| =  [0.0, 1.0]


And let's define a function that will tell us the NQR transition frequency for a particular atom and starting *m*.

In [3]:
def calc_nqr(atom, m):
    efg = atom.efg
    Q = atom.Q
    spin = atom.spin
    
    Vzz = efg.evals[2]
    vec_zz = efg.evecs[2]
    eta = (abs(efg.evals[0]) - abs(efg.evals[1]))/efg.evals[2]
    A = Vzz * (Q * millibarn) / (4.0 * spin * (2.0*spin - 1.0))
    fq = 3*A * (2.0*abs(m) + 1.0) * math.sqrt(1.0 + eta**2/3)

    return fq


Using these we can now loop over all chlorine nuclei in our system and calculate their NQR frequencies for all available transitions. Let's import our ab-initio EFG calculation's output .magres file,

In [4]:
atoms = MagresAtoms.load_magres('../samples/NaClO3.magres')

and then perform the calculation:

In [5]:
freqs = []

for Cl_atom in atoms.species('Cl'):
    print(Cl_atom, "S={:.1f} Q={:.2f} millibarn".format(Cl_atom.spin, Cl_atom.Q))
    
    for m in available_m(Cl_atom.spin):
        freq_MHz = calc_nqr(Cl_atom, m) / megahertz
        print("  m={:.1f}->{:.1f} freq = {:.3f} MHz".format(m, m+1, freq_MHz))
        
        freqs.append(freq_MHz)
        
print("Mean freq = {:.3f} Mhz".format(numpy.mean(freqs)))

35Cl1 S=1.5 Q=-81.65 millibarn
  m=0.5->1.5 freq = 28.844 MHz
35Cl2 S=1.5 Q=-81.65 millibarn
  m=0.5->1.5 freq = 29.057 MHz
35Cl3 S=1.5 Q=-81.65 millibarn
  m=0.5->1.5 freq = 28.844 MHz
35Cl4 S=1.5 Q=-81.65 millibarn
  m=0.5->1.5 freq = 28.844 MHz
Mean freq = 28.898 Mhz


The mean frequency is close to the experimental value of ~30 Mhz.